In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('text_emotion-ori.csv')#Read data set from local file

In [3]:
data.head(3)#See the first three data sets

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...


In [4]:
data.groupby(['sentiment']).count()#Probably look at the data set classification number

,tweet_id,author,content
sentiment,,,
anger,111,111,111
boredom,180,180,180
empty,831,831,831
enthusiasm,754,754,754
fun,1776,1776,1776
happiness,5209,5209,5209
hate,1323,1323,1323
love,3842,3842,3842
neutral,8638,8638,8638


In [5]:
data = data.replace('anger', 0)
data = data.replace('boredom', 1)
data = data.replace('empty', 3)
data = data.replace('enthusiasm', 3)
data = data.replace('fun', 2)
data = data.replace('happiness', 2)
data = data.replace('hate', 0)
data = data.replace('love', 2)
data = data.replace('neutral', 3)
data = data.replace('relief', 3)
data = data.replace('sadness', 1)
data = data.replace('surprise', 0)
data = data.replace('worry', 1)
#Reclassify, merge 13 categories into 4 categories, improve accuracy

In [6]:
data.head(3)

,tweet_id,sentiment,author,content
0,1956967341,3,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,1,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,1,coolfunky,Funeral ceremony...gloomy friday...


In [7]:
symbol='??,???,????,?????,??????,???????,????????,?????????,??????????,???????????,????????????,???????????????,!!,!!!,!!!!,!!!!!,!!!!!!,!!!!!!!,!!!!!!!!,!!!!!!!!!,!!!!!!!!!!,!!!!!!!!!!!,!!!!!!!!!!!!,!!!!!!!!!!!!!!!,?!!!!!!!!!!!!!!,!?,?!,!??,?!?,??!,!??!,((((((((((((((((((,xx,xxx,xxxx,xxxxx,xxxxxx,xxxxxxx,xxxxxxxx,xxxxxxxxx,xxxxxxxxxx,xxxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxxxxxxxx,X,XX,Xx,Xxxx,xoxo,XxOo,XOXO,ox,xox,Xoxo,oxox,xo,xoxox,xxoo,XoxoX,33,333,3333,$$,$$$,=),=(,=D,=X,=x,=C,=c,=P,=p,=>,=<,=[,=],=|,=#,=o,=O,=/,=\,=3,=S,=s,=0,:),:(,:D,:X,:x,:C,:c,:P,:p,:>,:<,:[,:],:|,:#,:o,:O,:/,:\,:3,:S,:s,:0,;),;(,;D,;X,;x,;C,;c,;P,;p,;>,;<,;[,;],;|,;#,;o,;O,;/,;\,;3,;S,;s,;0,;-),;-(,;-D,;-X,;-x,;-C,;-c,;-P,;-p,;->,;-<,;-[,;-],;-|,;-#,;-o,;-O,;-/,;-\,;-3,;-S,;-s,;0,:-),:-(,:-D,:-X,:-x,:-C,:-c,:-P,:-p,:->,:-<,:-[,:-],:-|,:-#,:-o,:-O,:-/,:-\,:-3,:-S,:-s,:-0,X),X(,XD,XX,Xx,XC,Xc,XP,Xp,X>,X<,X[,X],X|,X#,Xo,XO,X/,X\,X3,XS,Xs,X0,x),x(,xD,xx,xx,xC,xc,xP,xp,x>,x<,x[,x],x|,x#,xo,xO,x/,x\,x3,xS,xs,x0,:\'(,:\'),:0),:^),;)),:-bd,:**(,:**-(,:-,:l,:-ss,:L,;))),D;,):,(:,D:,8(,=.=,o_O,0_o,O_O,;_;,~*~,*-*,(*    *),^.^,(^~^),x.x,-_-,-.-,:*,:*:*,((.,???-??,!),WOOOO,*|)|/-\|\|/-\*,!!!! /\/\/\/\/\@-),*lol*,(_!_)!!'
symbols = symbol.split(',')
#We enumerated 40,000 pieces of data that are more important for expressing emotions. In the following filters, these symbols are extracted

In [8]:
import nltk
# count all the words in the text
import re,collections  
def get_words(file):   
    words_box=[]  
    for line in file:
        line = line.lower()
        tokens = nltk.word_tokenize(line)
        words_box.extend(tokens)                 
    return collections.Counter(words_box) 
#The word frequency of each word is counted here for the filter

In [9]:
word_cnt = get_words(data.content)#Content is the content and body of the comment

In [10]:
# get the category of the word by using nltk method
from nltk.corpus import stopwords
import re
tags = set(['CC','DT','MD','IN','NN', 'NNP', 'NNS', 'NNPS','UH', 'VB','VBD', 'VBG','MD', 'VBN', 'VBP', 'VBZ', 'RB', 'RBR', 'RBS', 'JJ', 'JJR', 'JJS'])
#Here uses the nltk POS_TAG method, which recognizes the part-of-speech of each word and leaves the above parts of words. Keep verbs, nouns, adjectives, adverbs, modal verbs, auxiliary verbs, and negative words.
def filter(text):#Filter method
    # drop all the stop word and the word only show 1 time
    #Stopwords and words that appear only once are deleted
    words = []
    for w in text.split():
        if w in stopwords.words('english'):
            continue
        if w in symbols:
            words.append(w)
            continue
        pat_letter = re.compile(r'[^a-zA-Z \']+')
        text = pat_letter.sub(' ', w).strip().lower()
        if text and text in word_cnt and word_cnt[w] > 1:
            words.append(text)
            
#     words = [w for w in text.split() if((w not in stopwords.words('english') and w in word_cnt and word_cnt[w] > 1) or w in symbols)]
    # get the categoyr of word
    pos_tags =nltk.pos_tag(words)
    ret = []
    #filter
    for word,pos in pos_tags:
        if (pos in tags):
            ret.append(word)
    return ' '.join(ret)

In [11]:
data['content'] = data['content'].apply(filter)
#Call the filter method to process the data set

In [12]:
data.head(3)

,tweet_id,sentiment,author,content
0,1956967341,3,xoshayzers,know listenin bad habit earlier started freaki...
1,1956967666,1,wannamama,n bed headache
2,1956967696,1,coolfunky,


In [13]:
data.to_csv("dataafterfilter.csv",index=False,sep=',')#Processed data output in dataafterfilter.csv